# ---------------------------------------------------------------------------------------------------------------
                                              Location in & out ID
                                               Final Version Code
# ---------------------------------------------------------------------------------------------------------------

# Importing the required packages & Establishing the Connection  with SQL

In [1]:
import os
import pyodbc
import pandas as pd
import numpy as np
import shutil
from datetime import date
import time
import sqlalchemy as sa
import urllib


server = 'tcp:sqlserverprod'
database = 'QualCommSatelliteData'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

server = 'tcp:sqlserverprod'
database = 'Reporting'
conn1 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor1 = conn1.cursor()

# Extracting the data From the SQL Server

In [2]:
QC1 = pd.read_sql_query(''' with cteEq as
(select v.EquipmentID
from QualCommSatelliteData.dbo.TruckQCEquipment_Vw v
),
cte as (
select max(A.QCVehiclePositionReportID_Entry) as QCEntry, A.EquipmentID
From Reporting.dvlpr.TruckGeofenceInOut A with (Nolock)
group by A.EquipmentID)

 
select cte.QCEntry,B.QCVehiclePositionReportID, B.EquipmentID, Convert(datetime,EventTS) as EventTSUTC, 
(case when FC.PolygonUpdatedBy is not null then C.FM_LocID else NULL end) FM_LocID
from QualCommSatelliteData.dbo.QCVehiclePositionReport B with (Nolock)
left join QualCommSatelliteData.dbo.QCVehicleLocation C with (Nolock) on B.QCVehiclePositionReportID = C.QCVehiclePositionReportID
left join FleetmanagerEx.dbo.Location FC with (Nolock) on C.FM_LocID=FC.LocID
left join cteEq on B.EquipmentID=cteEq.EquipmentID
Left Join cte on cte.EquipmentID = B.EquipmentID
where
(B.QCVehiclePositionReportID >= cte.QCEntry) or (cte.EquipmentID is NULL and B.EventTS >= DATEADD(day,-7, GETDATE())) ''', conn)

In [3]:
df = pd.DataFrame(QC1)

In [4]:
df.head(10)

,QCEntry,QCVehiclePositionReportID,EquipmentID,EventTSUTC,FM_LocID
0,235531541,237456140,18116,2021-01-25 21:46:59,NaN
1,235531541,237457108,18116,2021-01-25 21:46:59,NaN
2,235531541,237459496,18116,2021-01-25 21:46:59,NaN
3,235531541,237460308,18116,2021-01-25 21:46:59,NaN
4,235531541,237462255,18116,2021-01-25 21:46:59,NaN
5,235531541,237463207,18116,2021-01-25 21:46:59,NaN
6,235531541,237464110,18116,2021-01-25 21:46:59,NaN
7,235531541,237465983,18116,2021-01-25 21:46:59,NaN
8,235531541,237466867,18116,2021-01-25 21:46:59,NaN
9,235531541,237468925,18116,2021-01-25 21:46:59,NaN


##### 

In [5]:
df.shape

(234963, 5)

# Sorting the Values and Filling the Null Location as 10101010
# Reset the index 

In [6]:
df11= df.sort_values(['EquipmentID','EventTSUTC'], ascending=[True, True]).fillna(10101010).reset_index()

In [7]:
df11.head(5)

,index,QCEntry,QCVehiclePositionReportID,EquipmentID,EventTSUTC,FM_LocID
0,31474,259824634,259824634,19,2021-04-06 01:02:54,67567.0
1,28284,259791078,259791078,25,2021-04-05 22:39:35,10101010.0
2,29733,259791078,259793683,25,2021-04-05 22:51:17,10101010.0
3,29773,259791078,259794848,25,2021-04-05 22:54:36,10101010.0
4,30866,259791078,259798563,25,2021-04-05 23:09:21,10101010.0


In [8]:
df11.index

RangeIndex(start=0, stop=234963, step=1)

# Defining the For Loop With IF else Condition to Extract athe fist and last ID of each location 

In [9]:
output = []
index2 = 0
for index in df11.index:
        if index == 0:
            print('start')
        else:
            if (df11.loc[index,'EquipmentID']==df11.loc[index-1,'EquipmentID']) and (df11.loc[index,'FM_LocID']==df11.loc[index-1,'FM_LocID']):
                if (index2==0):
                    output.append(df11.loc[index-1,['EquipmentID','QCVehiclePositionReportID','EventTSUTC','FM_LocID']].to_list())
                    index2 += 1
            else:
                output.append(df11.loc[index-1,['EquipmentID','QCVehiclePositionReportID','EventTSUTC','FM_LocID']].to_list())
                index2 = 0
output

start


[[19, 259824634, Timestamp('2021-04-06 01:02:54'), 67567.0],
 [25, 259791078, Timestamp('2021-04-05 22:39:35'), 10101010.0],
 [25, 260045922, Timestamp('2021-04-06 16:47:06'), 10101010.0],
 [40, 231652384, Timestamp('2021-01-14 21:11:40'), 10101010.0],
 [40, 231657051, Timestamp('2021-01-14 21:27:05'), 10101010.0],
 [64, 246775166, Timestamp('2021-02-24 02:04:16'), 31390.0],
 [64, 253359419, Timestamp('2021-03-15 12:04:54'), 31390.0],
 [69, 236249516, Timestamp('2021-01-27 14:40:15'), 81905.0],
 [69, 236253911, Timestamp('2021-01-27 14:52:29'), 81905.0],
 [86, 256771051, Timestamp('2021-03-25 18:32:01'), 10101010.0],
 [86, 256816581, Timestamp('2021-03-25 20:54:46'), 10101010.0],
 [87, 259954932, Timestamp('2021-04-06 11:22:50'), 10101010.0],
 [87, 260046134, Timestamp('2021-04-06 16:50:42'), 10101010.0],
 [116, 259983304, Timestamp('2021-04-06 13:18:31'), 10101010.0],
 [116, 260041692, Timestamp('2021-04-06 16:34:38'), 10101010.0],
 [116, 260043052, Timestamp('2021-04-06 16:38:59'), 1

# Storing the Processed Data From List form to Data Frame

In [10]:
output_df = pd.DataFrame(output,columns=['EquipmentID','QCVehiclePositionReportID','EventTS_EST','LocId'])

In [11]:
output_df.head(10)

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId
0,19,259824634,2021-04-06 01:02:54,67567.0
1,25,259791078,2021-04-05 22:39:35,10101010.0
2,25,260045922,2021-04-06 16:47:06,10101010.0
3,40,231652384,2021-01-14 21:11:40,10101010.0
4,40,231657051,2021-01-14 21:27:05,10101010.0
5,64,246775166,2021-02-24 02:04:16,31390.0
6,64,253359419,2021-03-15 12:04:54,31390.0
7,69,236249516,2021-01-27 14:40:15,81905.0
8,69,236253911,2021-01-27 14:52:29,81905.0
9,86,256771051,2021-03-25 18:32:01,10101010.0


# --------------------------------------------------------------------------------------------------------------

# Taking copy of the data Frame

In [12]:
output_df1=output_df.copy()

# applying Lead Function to Shift one row From LocID and saving it as a new variable 

In [13]:
output_df1['LocId_lead']=output_df1['LocId'].shift(-1)

# applying Lead Function to Shift one row From EventTS_EST_lead and saving it as a new variable 

In [14]:
output_df1['EventTS_EST_lead']=output_df1['EventTS_EST'].shift(-1)

# applying Lead Function to Shift one row From QCVehiclePositionReportID_lead  and saving it as a new variable 

In [15]:
output_df1['QCVehiclePositionReportID_lead']=output_df1['QCVehiclePositionReportID'].shift(-1)

# Find the difference between LocLead and LocID 

In [16]:
output_df1['Diff']=output_df1['LocId_lead']-output_df1['LocId']

# Defining the for loop to Set flages as 0 and 1 for DIFF   

In [17]:
def shift(x):
    if x == 0:
        return 1
    else:
        return 0

In [18]:
output_df1['Shift_Flag']=output_df1['Diff'].apply(lambda x: shift(x))

# Defining the for loop to Set flages as 0 and 1 for EquipmentID_lead

In [19]:
def shiftFlag(x,y,z):
    if x==1 and y !=z:
        return 1
    else:
        return 0

In [20]:
output_df1['EquipmentID_lead']=output_df1['EquipmentID'].shift(-1)

In [21]:
output_df1['Flag']=output_df1.apply(lambda x: shiftFlag(x.Shift_Flag,x.EquipmentID,x.EquipmentID_lead),axis=1)

In [22]:
output_df1=output_df1[output_df1['Flag'] == 0]

In [23]:
output_df1.head(10)

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId,LocId_lead,EventTS_EST_lead,QCVehiclePositionReportID_lead,Diff,Shift_Flag,EquipmentID_lead,Flag
0,19,259824634,2021-04-06 01:02:54,67567.0,10101010.0,2021-04-05 22:39:35,259791078.0,10033443.0,0,25.0,0
1,25,259791078,2021-04-05 22:39:35,10101010.0,10101010.0,2021-04-06 16:47:06,260045922.0,0.0,1,25.0,0
3,40,231652384,2021-01-14 21:11:40,10101010.0,10101010.0,2021-01-14 21:27:05,231657051.0,0.0,1,40.0,0
4,40,231657051,2021-01-14 21:27:05,10101010.0,31390.0,2021-02-24 02:04:16,246775166.0,-10069620.0,0,64.0,0
5,64,246775166,2021-02-24 02:04:16,31390.0,31390.0,2021-03-15 12:04:54,253359419.0,0.0,1,64.0,0
6,64,253359419,2021-03-15 12:04:54,31390.0,81905.0,2021-01-27 14:40:15,236249516.0,50515.0,0,69.0,0
7,69,236249516,2021-01-27 14:40:15,81905.0,81905.0,2021-01-27 14:52:29,236253911.0,0.0,1,69.0,0
8,69,236253911,2021-01-27 14:52:29,81905.0,10101010.0,2021-03-25 18:32:01,256771051.0,10019105.0,0,86.0,0
9,86,256771051,2021-03-25 18:32:01,10101010.0,10101010.0,2021-03-25 20:54:46,256816581.0,0.0,1,86.0,0
11,87,259954932,2021-04-06 11:22:50,10101010.0,10101010.0,2021-04-06 16:50:42,260046134.0,0.0,1,87.0,0


# keeping all the 1 and removing 0 ie. all the multiple entries

In [24]:
output_df_final1=output_df1[output_df1['Shift_Flag'] == 1]

In [25]:
output_df_final1.head()

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId,LocId_lead,EventTS_EST_lead,QCVehiclePositionReportID_lead,Diff,Shift_Flag,EquipmentID_lead,Flag
1,25,259791078,2021-04-05 22:39:35,10101010.0,10101010.0,2021-04-06 16:47:06,260045922.0,0.0,1,25.0,0
3,40,231652384,2021-01-14 21:11:40,10101010.0,10101010.0,2021-01-14 21:27:05,231657051.0,0.0,1,40.0,0
5,64,246775166,2021-02-24 02:04:16,31390.0,31390.0,2021-03-15 12:04:54,253359419.0,0.0,1,64.0,0
7,69,236249516,2021-01-27 14:40:15,81905.0,81905.0,2021-01-27 14:52:29,236253911.0,0.0,1,69.0,0
9,86,256771051,2021-03-25 18:32:01,10101010.0,10101010.0,2021-03-25 20:54:46,256816581.0,0.0,1,86.0,0


In [26]:
Non_single=output_df_final1[['EquipmentID','LocId','QCVehiclePositionReportID','QCVehiclePositionReportID_lead','EventTS_EST','EventTS_EST_lead']]

In [27]:
Non_single.head()

,EquipmentID,LocId,QCVehiclePositionReportID,QCVehiclePositionReportID_lead,EventTS_EST,EventTS_EST_lead
1,25,10101010.0,259791078,260045922.0,2021-04-05 22:39:35,2021-04-06 16:47:06
3,40,10101010.0,231652384,231657051.0,2021-01-14 21:11:40,2021-01-14 21:27:05
5,64,31390.0,246775166,253359419.0,2021-02-24 02:04:16,2021-03-15 12:04:54
7,69,81905.0,236249516,236253911.0,2021-01-27 14:40:15,2021-01-27 14:52:29
9,86,10101010.0,256771051,256816581.0,2021-03-25 18:32:01,2021-03-25 20:54:46


# Renaming Non_Single entry variables names as required

In [28]:
Non_single.rename(columns = {'LocId':'FMLocId', 'QCVehiclePositionReportID':'QCVehiclePositionReportID_Entry', 
                             'QCVehiclePositionReportID_lead':'QCVehiclePositionReportID_Exit','EventTS_EST':'EventTS_Entry',
                             'EventTS_EST_lead':'EventTS_Exit'}, inplace = True)

C:\Users\Dinesh\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [29]:
Non_single.head(10)

,EquipmentID,FMLocId,QCVehiclePositionReportID_Entry,QCVehiclePositionReportID_Exit,EventTS_Entry,EventTS_Exit
1,25,10101010.0,259791078,260045922.0,2021-04-05 22:39:35,2021-04-06 16:47:06
3,40,10101010.0,231652384,231657051.0,2021-01-14 21:11:40,2021-01-14 21:27:05
5,64,31390.0,246775166,253359419.0,2021-02-24 02:04:16,2021-03-15 12:04:54
7,69,81905.0,236249516,236253911.0,2021-01-27 14:40:15,2021-01-27 14:52:29
9,86,10101010.0,256771051,256816581.0,2021-03-25 18:32:01,2021-03-25 20:54:46
11,87,10101010.0,259954932,260046134.0,2021-04-06 11:22:50,2021-04-06 16:50:42
13,116,10101010.0,259983304,260041692.0,2021-04-06 13:18:31,2021-04-06 16:34:38
16,118,10101010.0,260038552,260046022.0,2021-04-06 16:20:20,2021-04-06 16:48:41
18,166,10101010.0,260037231,260046067.0,2021-04-02 19:15:52,2021-04-06 16:49:25
20,174,68237.0,259037294,259042342.0,2021-04-02 01:17:17,2021-04-02 01:39:59


# Taking 2nd Copy From Copy data

In [30]:
output_df2=output_df.copy()

In [31]:
output_df2['LocId_lead']=output_df2['LocId'].shift(-1)
output_df2['LocId_lag']=output_df2['LocId'].shift(1)

In [32]:
output_df2.head(5)

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId,LocId_lead,LocId_lag
0,19,259824634,2021-04-06 01:02:54,67567.0,10101010.0,NaN
1,25,259791078,2021-04-05 22:39:35,10101010.0,10101010.0,67567.0
2,25,260045922,2021-04-06 16:47:06,10101010.0,10101010.0,10101010.0
3,40,231652384,2021-01-14 21:11:40,10101010.0,10101010.0,10101010.0
4,40,231657051,2021-01-14 21:27:05,10101010.0,31390.0,10101010.0


# Defining logic to get all the single entires out seperately

In [33]:
def equals(a,b,c):
    if a!=b and a!=c:
        return 1
    else:
        return 0

In [34]:
output_df2['Single_entry']=output_df2.apply(lambda x: equals(x.LocId,x.LocId_lead,x.LocId_lag),axis=1)

In [35]:
output_df_final2=output_df2[output_df2['Single_entry'] == 1]

In [36]:
output_df_final2.head(5)

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId,LocId_lead,LocId_lag,Single_entry
0,19,259824634,2021-04-06 01:02:54,67567.0,10101010.0,NaN,1
15,116,260043052,2021-04-06 16:38:59,16443.0,10101010.0,10101010.0,1
32,235,260009371,2021-04-06 14:48:27,100163.0,10101010.0,10101010.0,1
39,263,260042077,2021-04-06 16:34:18,9980.0,10101010.0,90900.0,1
67,555,247100011,2021-02-03 14:58:44,1.0,68237.0,10101010.0,1


# taking all the single entry values 

In [37]:
single=output_df_final2[['EquipmentID','QCVehiclePositionReportID','EventTS_EST','LocId']]

# marking down lead lag To find the continuation of QCVehiclePositionReportID

In [38]:
single['QCVehiclePositionReportID_lead']=single['QCVehiclePositionReportID']

<ipython-input-38-bc3892a9a6f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single['QCVehiclePositionReportID_lead']=single['QCVehiclePositionReportID']


# marking down lead lag To find the continuation of EventTS_EST

In [39]:
single['EventTS_EST_lead']=single['EventTS_EST']

<ipython-input-39-2b0038f66aee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single['EventTS_EST_lead']=single['EventTS_EST']


In [40]:
single.head()

,EquipmentID,QCVehiclePositionReportID,EventTS_EST,LocId,QCVehiclePositionReportID_lead,EventTS_EST_lead
0,19,259824634,2021-04-06 01:02:54,67567.0,259824634,2021-04-06 01:02:54
15,116,260043052,2021-04-06 16:38:59,16443.0,260043052,2021-04-06 16:38:59
32,235,260009371,2021-04-06 14:48:27,100163.0,260009371,2021-04-06 14:48:27
39,263,260042077,2021-04-06 16:34:18,9980.0,260042077,2021-04-06 16:34:18
67,555,247100011,2021-02-03 14:58:44,1.0,247100011,2021-02-03 14:58:44


# Renaming Single entry variables names as required

In [41]:
single.rename(columns = {'LocId':'FMLocId', 'QCVehiclePositionReportID':'QCVehiclePositionReportID_Entry', 
                             'QCVehiclePositionReportID_lead':'QCVehiclePositionReportID_Exit','EventTS_EST':'EventTS_Entry',
                             'EventTS_EST_lead':'EventTS_Exit'}, inplace = True)

C:\Users\Dinesh\Anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


# storing Renamed Single values as a data frame

In [42]:
single=single[['EquipmentID','FMLocId','QCVehiclePositionReportID_Entry','QCVehiclePositionReportID_Exit','EventTS_Entry','EventTS_Exit']]

In [43]:
single.head(5)

,EquipmentID,FMLocId,QCVehiclePositionReportID_Entry,QCVehiclePositionReportID_Exit,EventTS_Entry,EventTS_Exit
0,19,67567.0,259824634,259824634,2021-04-06 01:02:54,2021-04-06 01:02:54
15,116,16443.0,260043052,260043052,2021-04-06 16:38:59,2021-04-06 16:38:59
32,235,100163.0,260009371,260009371,2021-04-06 14:48:27,2021-04-06 14:48:27
39,263,9980.0,260042077,260042077,2021-04-06 16:34:18,2021-04-06 16:34:18
67,555,1.0,247100011,247100011,2021-02-03 14:58:44,2021-02-03 14:58:44


# Joining single and NoN single values 

In [44]:
Final=Non_single.append(single)

In [45]:
Final

,EquipmentID,FMLocId,QCVehiclePositionReportID_Entry,QCVehiclePositionReportID_Exit,EventTS_Entry,EventTS_Exit
1,25,10101010.0,259791078,260045922.0,2021-04-05 22:39:35,2021-04-06 16:47:06
3,40,10101010.0,231652384,231657051.0,2021-01-14 21:11:40,2021-01-14 21:27:05
5,64,31390.0,246775166,253359419.0,2021-02-24 02:04:16,2021-03-15 12:04:54
7,69,81905.0,236249516,236253911.0,2021-01-27 14:40:15,2021-01-27 14:52:29
9,86,10101010.0,256771051,256816581.0,2021-03-25 18:32:01,2021-03-25 20:54:46
...,...,...,...,...,...,...
4370,18483,10101010.0,260044612,260044612.0,2021-04-06 16:45:24,2021-04-06 16:45:24
4371,18484,81905.0,260040089,260040089.0,2021-04-06 16:28:58,2021-04-06 16:28:58
4372,18484,10101010.0,260044538,260044538.0,2021-04-06 16:44:00,2021-04-06 16:44:00
4373,18485,81905.0,260041594,260041594.0,2021-04-06 16:33:09,2021-04-06 16:33:09


# After Appending the Two Data frame , sorting it by EventTS_Entry and Equipmentid 

In [46]:
Final = Final.sort_values(['EquipmentID','EventTS_Entry'], ascending=[True, True])

In [47]:
Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332 entries, 0 to 4375
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   EquipmentID                      2332 non-null   int64         
 1   FMLocId                          2332 non-null   float64       
 2   QCVehiclePositionReportID_Entry  2332 non-null   int64         
 3   QCVehiclePositionReportID_Exit   2332 non-null   float64       
 4   EventTS_Entry                    2332 non-null   datetime64[ns]
 5   EventTS_Exit                     2332 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 127.5 KB


In [48]:
Final.head(10)

,EquipmentID,FMLocId,QCVehiclePositionReportID_Entry,QCVehiclePositionReportID_Exit,EventTS_Entry,EventTS_Exit
0,19,67567.0,259824634,259824634.0,2021-04-06 01:02:54,2021-04-06 01:02:54
1,25,10101010.0,259791078,260045922.0,2021-04-05 22:39:35,2021-04-06 16:47:06
3,40,10101010.0,231652384,231657051.0,2021-01-14 21:11:40,2021-01-14 21:27:05
5,64,31390.0,246775166,253359419.0,2021-02-24 02:04:16,2021-03-15 12:04:54
7,69,81905.0,236249516,236253911.0,2021-01-27 14:40:15,2021-01-27 14:52:29
9,86,10101010.0,256771051,256816581.0,2021-03-25 18:32:01,2021-03-25 20:54:46
11,87,10101010.0,259954932,260046134.0,2021-04-06 11:22:50,2021-04-06 16:50:42
13,116,10101010.0,259983304,260041692.0,2021-04-06 13:18:31,2021-04-06 16:34:38
15,116,16443.0,260043052,260043052.0,2021-04-06 16:38:59,2021-04-06 16:38:59
16,118,10101010.0,260038552,260046022.0,2021-04-06 16:20:20,2021-04-06 16:48:41


# Final Data Frame with single and multiple entry combained

In [49]:
Final

,EquipmentID,FMLocId,QCVehiclePositionReportID_Entry,QCVehiclePositionReportID_Exit,EventTS_Entry,EventTS_Exit
0,19,67567.0,259824634,259824634.0,2021-04-06 01:02:54,2021-04-06 01:02:54
1,25,10101010.0,259791078,260045922.0,2021-04-05 22:39:35,2021-04-06 16:47:06
3,40,10101010.0,231652384,231657051.0,2021-01-14 21:11:40,2021-01-14 21:27:05
5,64,31390.0,246775166,253359419.0,2021-02-24 02:04:16,2021-03-15 12:04:54
7,69,81905.0,236249516,236253911.0,2021-01-27 14:40:15,2021-01-27 14:52:29
...,...,...,...,...,...,...
4371,18484,81905.0,260040089,260040089.0,2021-04-06 16:28:58,2021-04-06 16:28:58
4372,18484,10101010.0,260044538,260044538.0,2021-04-06 16:44:00,2021-04-06 16:44:00
4373,18485,81905.0,260041594,260041594.0,2021-04-06 16:33:09,2021-04-06 16:33:09
4374,18485,10101010.0,260046019,260046019.0,2021-04-06 16:48:35,2021-04-06 16:48:35


# Loading / Inserting the  Final data frame into SQl Temp table

In [50]:
server = 'tcp:sqlserverprod'
database = 'Reporting'
conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn))
Final.to_sql("TruckGeofenceInOut_Temp", engine, schema='dvlpr', chunksize=200, method='multi', index=False, if_exists='replace')

In [51]:
conn1.commit()

# Calling the stored procedure in Sql to compare the data from temp table and main table and update the new values in the main table.

In [52]:
cursor1.execute("Exec [dvlpr].[DelUpdate_source]")

In [53]:
conn1.commit()

# Once update values into main table delete the values in the temp table

In [54]:
cursor1.execute("DELETE FROM Reporting.dvlpr.TruckGeofenceInOut_Temp")

In [55]:
conn1.commit()

# closing all the Connection and cursors Established

In [56]:
cursor.close()
cursor1.close()

In [57]:
conn1.close()

# ---------------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------